In [ ]:
f_dir = '/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/Official_NMSSM'
mass_point = 'NMSSM_XToYHTo6B_MX-700_MY-400_TuneCP5_13TeV-madgraph-pythia8'

model_dir = '/eos/uscms/store/user/srosenzw/weaver/models/exp_sixb_official/feynnet_ranker_6b/20230731_7d266883bbfb88fe4e226783a7d1c9db_ranger_lr0.0047_batch2000_withbkg/predict_output/2018'

In [ ]:
import subprocess
output = subprocess.check_output(f'ls {model_dir}/*_NMSSM*', shell=True).decode('utf-8').split('\n')
output = [out for out in output if 'eos' in out]

In [ ]:
systematics = [out.split('/')[-1].split('_NMSSM')[0].split('_') for out in output]
systematics = [['_'.join(sys[:-1]), sys[-1]] if len(sys) == 3 else sys for sys in systematics]
systematics

In [ ]:
sys_dict = {
    'Absolute_2018' : 'bSFshape_X_jesAbsolute',
    'Absolute' : 'bSFshape_X_jesAbsolute',
    'BBEC1_2018' : 'bSFshape_X_jesBBEC1',
    'BBEC1' : 'bSFshape_X_jesBBEC1',
    'EC2_2018' : 'bSFshape_X_jesEC2',
    'EC2' : 'bSFshape_X_jesEC2',
    'FlavorQCD' : 'bSFshape_X_jesFlavorQCD',
    'HF_2018' : 'bSFshape_X_jesHF',
    'HF' : 'bSFshape_X_jesHF',
    'RelativeBal' : 'bSFshape_X_jesRelativeBal',
    'RelativeSample_2018' : 'bSFshape_X_jesRelativeSample',
}

In [ ]:
mx_dict = {}

In [ ]:
nominal = SixB(f"{f_dir}/{mass_point}/ntuple.root", feyn=False)

# fig, ax = plt.subplots()    
# mBins = np.linspace(375,1500,31)
# n_nominal = Hist(nominal.X.m, bins=mBins, weights=nominal.scale, ax=ax)
# n_central = Hist(nominal.X.m, bins=mBins, weights=nominal.scale*nominal.bSFshape_central, ax=ax)

# mx_dict['nominal'] = n_nominal
# mx_dict['bSFshape_central'] = n_central

In [ ]:

for sys in systematics:
    if 'Total' in sys[0]: continue
    sys_name_up = sys_dict[sys[0]].replace('X', 'up')
    sys_name_down = sys_dict[sys[0]].replace('X', 'down')
    # if sys_name_up in mx_dict.keys(): continue

    tree = SixB(f"{f_dir}/syst/{sys[0]}/{sys[1]}/{mass_point}/ntuple.root", feyn=f'{model_dir}/{sys[0]}_{sys[1]}_{mass_point}.root')

    sys_up = tree.get(sys_name_up)
    sys_down = tree.get(sys_name_down)

    fig, ax = plt.subplots()    
    mBins = np.linspace(375,1500,31)
    n_up = Hist(tree.X.m, bins=mBins, weights=tree.scale*sys_up, ax=ax)
    n_down = Hist(tree.X.m, bins=mBins, weights=tree.scale*sys_down, ax=ax)

    mx_dict[sys_name_up] = n_up
    mx_dict[sys_name_down] = n_down

    plt.close()


In [ ]:
mx_dict

In [ ]:
sys_list = ['Absolute_2018', 'Absolute', 'BBEC1_2018', 'BBEC1', 'EC2_2018', 'EC2', 'FlavorQCD', 'HF_2018', 'HF', 'RelativeBal', 'RelativeSample_2018']

In [ ]:
x = (mBins[1:] + mBins[:-1])/2

In [ ]:
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec

In [ ]:
fig = plt.figure(figsize=(30,30))
gs = GridSpec(4, 3, figure=fig)

for sys,ss in zip(sys_list, gs):
    gs0 = GridSpecFromSubplotSpec(4, 1, subplot_spec=ss, hspace=0)

    ax1 = fig.add_subplot(gs0[:-1])
    ax2 = fig.add_subplot(gs0[-1], sharex=ax1)

    sys_up = sys_dict[sys].replace('X', 'up')
    sys_down = sys_dict[sys].replace('X', 'down')

    n = Hist(x, bins=mBins, weights=mx_dict['bSFshape_central'], ax=ax1, label='nominal', color='k')
    n_up = Hist(x, bins=mBins, weights=mx_dict[sys_up], ax=ax1, label=f"{sys}_up", color='b')
    n_down = Hist(x, bins=mBins, weights=mx_dict[sys_down], ax=ax1, label=f"{sys}_down", color='r')

    ratio_up = np.nan_to_num(n_up/n, 1.0)
    ratio_down = np.nan_to_num(n_down/n, 1.0)

    n_ratio_up = Hist(x, bins=mBins, weights=ratio_up, ax=ax2, color='b')
    n_ratio_down = Hist(x, bins=mBins, weights=ratio_down, ax=ax2, color='r')
    ax2.plot([375,1500], [1,1], 'k--')
    ax2.set_ylim(0.8,1.2)
    ax2.set_ylabel('Ratio')
    ax2.set_xlabel(r'$m_{X}$ [GeV]')
    ax1.set_ylabel('Events')
    ax1.legend()

# gs0 = GridSpecFromSubplotSpec(4, 1, subplot_spec=gs[-1], hspace=0)
# ax1 = fig.add_subplot(gs0[:-1])
# ax2 = fig.add_subplot(gs0[-1], sharex=ax1)
# n = Hist(x, bins=mBins, weights=mx_dict['nominal'], ax=ax1, label='nominal', color='k')
# n_c = Hist(x, bins=mBins, weights=mx_dict['bSFshape_central'], ax=ax1, label="central", color='b')
# ratio = np.nan_to_num(n_c/n, 1.0)
# n_ratio = Hist(x, bins=mBins, weights=ratio, ax=ax2, color='b')
# ax2.plot([375,1500], [1,1], 'k--')
# ax2.set_ylim(0.5,1.5)
# ax2.set_ylabel('Ratio')
# ax2.set_xlabel(r'$m_{X}$ [GeV]')
# ax1.set_ylabel('Events')
# ax1.legend()

fig.savefig('plots/systematics/up_down.pdf')

In [ ]:
for sys,var in systematics:
    print(f"{sys}{var.capitalize()}")

In [ ]:
nominal.asr_hs_mask.sum()*nominal.scale

In [ ]:
import uproot
f = uproot.open("/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/Official_NMSSM/NMSSM_XToYHTo6B_MX-850_MY-350_TuneCP5_13TeV-madgraph-pythia8/ntuple.root")
t = f['sixBtree']
ttbar = uproot.open("/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/TTJets/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/ntuple.root")
ttbar = f['sixBtree']
tt_genWeight = ttbar['genWeight'].array()

In [ ]:
t.keys()

In [ ]:
import uproot
f = uproot.open("/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/Official_NMSSM/NMSSM_XToYHTo6B_MX-850_MY-350_TuneCP5_13TeV-madgraph-pythia8/ntuple.root")
print(f.keys())
t = f['sixBtree']
t['genWeight'].array(), len(t['jet_btag'].array()), [key for key in t.keys() if 'bSFshape' in key]

In [ ]:
import uproot
f = uproot.open("/uscms/home/srosenzw/nobackup/workarea/higgs/sixb_analysis/CMSSW_10_6_19_patch2/src/MultiHiggs/analysis/MultiHAnalysis/output.root")
print(f.keys())
t = f['sixBtree']
t['genWeight'].array(), len(t['jet_btag'].array())

In [ ]:
import uproot
f = uproot.open("/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/TTJets/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/ntuple.root")
t = f['sixBtree']
t['genWeight'].array()

In [ ]:
ttbar = Bkg("/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/TTJets/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/ntuple.root")
# tt_weight = ttbar.genWeight * ttbar.PUWeight * ttbar.triggerSF
# tt_pu_up = ttbar.genWeight * ttbar.PUWeight_up * ttbar.triggerSF
# tt_pu_down = ttbar.genWeight * ttbar.PUWeight_down * ttbar.triggerSF
# tt_trigger_up = ttbar.genWeight * ttbar.PUWeight * ttbar.triggerSF_up
# tt_trigger_down = ttbar.genWeight * ttbar.PUWeight * ttbar.triggerSF_down

tt_pu = ttbar.PUWeight
tt_pu_up = ttbar.PUWeight_up
tt_pu_down = ttbar.PUWeight_down

In [ ]:
ttbar.nevents

In [ ]:
fig, ax = plt.subplots()

n = Hist(ttbar.X.m, bins=np.linspace(375,1500,25), ax=ax, weights=tt_pu, label='nominal', color='k')
n_pu_up = Hist(ttbar.X.m, bins=np.linspace(375,1500,25), weights=tt_pu_up, ax=ax, label='pu_up', color='b')
n_pu_down = Hist(ttbar.X.m, bins=np.linspace(375,1500,25), weights=tt_pu_down, ax=ax, label='pu_down', color='r')

In [ ]:
import uproot
f = uproot.open("combine/feynnet/20230731_7d266883bbfb88fe4e226783a7d1c9db_ranger_lr0.0047_batch2000_withbkg/MX_850_MY_350.root")
f.keys()

In [ ]:
n, bins = f['signal'].to_numpy()
n_trig_up = f['signal_TriggerUp'].to_numpy()[0]
n_trig_down = f['signal_TriggerDown'].to_numpy()[0]
n_pu_up = f['signal_PileupUp'].to_numpy()[0]
n_pu_down = f['signal_PileupDown'].to_numpy()[0]
n_hf_up = f['signal_BTagHFUp'].to_numpy()[0]
n_hf_down = f['signal_BTagHFDown'].to_numpy()[0]
n_hfstats1_up = f['signal_BTagHFStats1Up'].to_numpy()[0]
n_hfstats1_down = f['signal_BTagHFStats1Down'].to_numpy()[0]
n_hfstats2_up = f['signal_BTagHFStats2Up'].to_numpy()[0]
n_hfstats2_down = f['signal_BTagHFStats2Down'].to_numpy()[0]
n_lf_up = f['signal_BTagLFUp'].to_numpy()[0]
n_lf_down = f['signal_BTagLFDown'].to_numpy()[0]
n_lfstats1_up = f['signal_BTagLFStats1Up'].to_numpy()[0]
n_lfstats1_down = f['signal_BTagLFStats1Down'].to_numpy()[0]
n_lfstats2_up = f['signal_BTagLFStats2Up'].to_numpy()[0]
n_lfstats2_down = f['signal_BTagLFStats2Down'].to_numpy()[0]
x = (bins[1:] + bins[:-1])/2

In [ ]:
fig, ax = plt.subplots()

n = Hist(x, bins=bins, weights=n, ax=ax, label='nominal', color='k')
n = Hist(x, bins=bins, weights=n_trig_up, ax=ax, label="trigger up", color='C0')
n = Hist(x, bins=bins, weights=n_trig_down, ax=ax, label="trigger down", color='C1')

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20,10))

n = Hist(x, bins=bins, weights=n, ax=ax, label='nominal', color='k')
n = Hist(x, bins=bins, weights=n_pu_up, ax=ax, label="pileup up", color='C0')
n = Hist(x, bins=bins, weights=n_pu_down, ax=ax, label="pileup down", color='C1')



In [ ]:
fig, ax = plt.subplots()

n = Hist(x, bins=bins, weights=n, ax=ax, label='nominal', color='k')
n = Hist(x, bins=bins, weights=n_hf_up, ax=ax, label="btag hf up", color='C0')
n = Hist(x, bins=bins, weights=n_hf_down, ax=ax, label="btag hf down", color='C1')

In [ ]:
fig, ax = plt.subplots()

n = Hist(x, bins=bins, weights=n, ax=ax, label='nominal', color='k')
n = Hist(x, bins=bins, weights=n_hfstats1_up, ax=ax, label="btag hfstats1 up", color='C0')
n = Hist(x, bins=bins, weights=n_hfstats1_down, ax=ax, label="btag hfstats1 down", color='C1')

In [ ]:
fig, ax = plt.subplots()

n = Hist(x, bins=bins, weights=n, ax=ax, label='nominal', color='k')
n = Hist(x, bins=bins, weights=n_hfstats2_up, ax=ax, label="btag hfstats2 up", color='C0')
n = Hist(x, bins=bins, weights=n_hfstats2_down, ax=ax, label="btag hfstats2 down", color='C1')